In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.metrics import roc_auc_score
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgb

In [3]:
delays = pd.read_excel("Простои за июль 2024.xlsx")
sensor_data = pd.read_excel("Датчики за июль 2024.xlsx")

In [3]:
delays.head()

,start,end,duration,downtime,reason,description,reason_id,detailed_description,component
0,2024-07-01 11:35:08,2024-07-01 11:55:33,20.416666,Переналадки,Переход продукт вода продукт,Переход продукт вода продукт + АПМ,2214102,переход,Розлив
1,2024-07-01 15:52:59,2024-07-01 16:14:41,21.700000,Переналадки,Переход продукт вода продукт,Переход продукт вода продукт + АПМ,2214102,переход,Розлив
2,2024-07-01 17:52:22,2024-07-01 18:21:48,29.433333,Прочие внеплановые простои,Загруженность склада,Съем ГП,2112001,нет сьема,Розлив
3,2024-07-01 18:22:22,2024-07-01 18:31:06,8.733333,Прочие внеплановые простои,Загруженность склада,Съем ГП,2112001,нет сьема,Розлив
4,2024-07-01 18:51:42,2024-07-01 18:54:03,2.350000,Прочие внеплановые простои,Загруженность склада,Съем ГП,2112001,нет сьема,Розлив


In [4]:
sensor_data.head()

,start,end,duration,reason
0,2024-07-01 08:00:00,2024-07-01 08:13:00,13.400000,Производство > 95%
1,2024-07-01 08:13:00,2024-07-01 08:13:00,0.183333,Вмешательство оператора\n
2,2024-07-01 08:13:00,2024-07-01 08:18:00,4.433333,Производство > 75% < 95%
3,2024-07-01 08:18:00,2024-07-01 09:17:00,59.100000,Производство > 95%
4,2024-07-01 09:17:00,2024-07-01 09:18:00,1.016666,Внешняя ошибка


In [10]:
delays.info(), sensor_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   start                 346 non-null    datetime64[ns]
 1   end                   346 non-null    datetime64[ns]
 2   duration              346 non-null    float64       
 3   downtime              346 non-null    object        
 4   reason                346 non-null    object        
 5   description           346 non-null    object        
 6   reason_id             346 non-null    int64         
 7   detailed_description  346 non-null    object        
 8   component             346 non-null    object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(5)
memory usage: 24.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6644 entries, 0 to 6643
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    ---------

(None, None)

In [15]:
sensor_data["start"] = sensor_data["start"].apply(lambda x: x.replace(second=0))
sensor_data["end"] = sensor_data["end"].apply(lambda x: x.replace(second=0))

delays["start"] = delays["start"].apply(lambda x: x.replace(second=0))
delays["end"] = delays["end"].apply(lambda x: x.replace(second=0))

In [ ]:
delay_full_data = pd.merge(sensor_data, delays, how="inner", on=["start", "end", "duration"])
delay_full_data

,start,end,duration,reason_x,downtime,reason_y,description,reason_id,detailed_description,component
0,2024-07-01 11:35:00,2024-07-01 11:55:00,20.416666,Вмешательство оператора\n,Переналадки,Переход продукт вода продукт,Переход продукт вода продукт + АПМ,2214102,переход,Розлив
1,2024-07-01 15:52:00,2024-07-01 16:14:00,21.700000,Вмешательство оператора\n,Переналадки,Переход продукт вода продукт,Переход продукт вода продукт + АПМ,2214102,переход,Розлив
2,2024-07-01 17:52:00,2024-07-01 18:21:00,29.433333,Затор на выходе,Прочие внеплановые простои,Загруженность склада,Съем ГП,2112001,нет сьема,Розлив
3,2024-07-01 18:22:00,2024-07-01 18:31:00,8.733333,Затор на выходе,Прочие внеплановые простои,Загруженность склада,Съем ГП,2112001,нет сьема,Розлив
4,2024-07-01 18:51:00,2024-07-01 18:54:00,2.350000,Вмешательство оператора\n,Прочие внеплановые простои,Загруженность склада,Съем ГП,2112001,нет сьема,Розлив
...,...,...,...,...,...,...,...,...,...,...
341,2024-07-30 19:11:00,2024-07-30 19:15:00,4.383333,Внешняя ошибка,Санитарные регламенты,Внешние мойки,Ополаскивание укупора и розлива,2310502,Обл.гор. водой,Автомат укупорочный
342,2024-07-30 21:05:00,2024-07-30 21:09:00,3.916666,Внешняя ошибка,Организационные,Проверки по качеству,Лабораторные проверки,2512701,Отбор проб.,Розлив
343,2024-07-30 21:42:00,2024-07-31 00:08:00,146.950000,Нехватка на входе,Переналадки,Переход через щ. СИП,Продукт СИП щ. продукт,2214201,Переход на FLash. СИП. АПМ.,Розлив
344,2024-07-31 00:09:00,2024-07-31 00:11:00,2.350000,Вмешательство оператора\n,Переналадки,Переход через щ. СИП,Продукт СИП щ. продукт,2214201,Переход на FLash. СИП. АПМ.,Розлив


In [ ]:
delay_full_data["reason_x"].value_counts()

reason_x
Вмешательство оператора\n    189
Внешняя ошибка                82
Нехватка на входе             37
Затор на выходе               14
Останов                       12
Ручное управление              6
Аварийная остановка            5
Внутренняя ошибка\n            1
Name: count, dtype: int64

In [36]:
delay_full_data["reason_y"].value_counts()

reason_y
Переход продукт вода продукт          81
Внешние мойки                         79
Переход через щ. СИП                  43
Проверки по качеству                  23
Перерывы                              21
Отсутствие заявок                     20
Переход продукт на продукт            17
Комментируемые остановки              13
Смена бутылки/банки                   13
Переход через стерил.                  8
Смена этикетки                         8
ПТО                                    6
Загруженность склада                   5
Тесты материалов                       3
Недостаточное количество персонала     2
Втор. упак.                            2
Переход через щ-к СИП                  1
Поддержка производства                 1
Name: count, dtype: int64

In [37]:
delay_full_data[delay_full_data["reason_x"] == "Аварийная остановка"]["reason_y"].value_counts()

reason_y
Внешние мойки               2
Комментируемые остановки    2
Переход через щ. СИП        1
Name: count, dtype: int64

In [38]:
delay_outer_data = pd.merge(sensor_data, delays, how="outer", on=["start", "end", "duration"])
delay_outer_data

,start,end,duration,reason_x,downtime,reason_y,description,reason_id,detailed_description,component
0,2024-07-01 08:00:00,2024-07-01 08:13:00,13.400000,Производство > 95%,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-07-01 08:13:00,2024-07-01 08:13:00,0.183333,Вмешательство оператора\n,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-07-01 08:13:00,2024-07-01 08:18:00,4.433333,Производство > 75% < 95%,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-07-01 08:18:00,2024-07-01 09:17:00,59.100000,Производство > 95%,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-07-01 09:17:00,2024-07-01 09:18:00,1.016666,Внешняя ошибка,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6639,2024-07-31 00:18:00,2024-07-31 00:48:00,30.016666,Производство > 95%,NaN,NaN,NaN,NaN,NaN,NaN
6640,2024-07-31 00:48:00,2024-07-31 00:50:00,2.000000,Производство > 75% < 95%,NaN,NaN,NaN,NaN,NaN,NaN
6641,2024-07-31 00:50:00,2024-07-31 01:21:00,31.516666,Производство > 95%,NaN,NaN,NaN,NaN,NaN,NaN
6642,2024-07-31 01:21:00,2024-07-31 01:22:00,0.316666,Производство > 75% < 95%,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
delay_full_data.to_excel("Сконкатенированные простои.xlsx")

In [43]:
delay_full_data[delay_full_data["reason_x"] == "Вмешательство оператора\n"]

,start,end,duration,reason_x,downtime,reason_y,description,reason_id,detailed_description,component
0,2024-07-01 11:35:00,2024-07-01 11:55:00,20.416666,Вмешательство оператора\n,Переналадки,Переход продукт вода продукт,Переход продукт вода продукт + АПМ,2214102,переход,Розлив
1,2024-07-01 15:52:00,2024-07-01 16:14:00,21.700000,Вмешательство оператора\n,Переналадки,Переход продукт вода продукт,Переход продукт вода продукт + АПМ,2214102,переход,Розлив
4,2024-07-01 18:51:00,2024-07-01 18:54:00,2.350000,Вмешательство оператора\n,Прочие внеплановые простои,Загруженность склада,Съем ГП,2112001,нет сьема,Розлив
7,2024-07-01 22:33:00,2024-07-01 23:17:00,44.000000,Вмешательство оператора\n,Переналадки,Переход продукт вода продукт,Переход продукт вода продукт + АПМ,2214102,переход на Флэш манго.,Розлив
8,2024-07-01 23:57:00,2024-07-02 00:00:00,2.200000,Вмешательство оператора\n,Организационные,Перерывы,Прогон в 00:00 часов,2511202,прогон партии.,Розлив
...,...,...,...,...,...,...,...,...,...,...
334,2024-07-29 23:55:00,2024-07-30 00:16:00,20.833333,Вмешательство оператора\n,Организационные,Перерывы,Прогон в 00:00 часов,2511202,бакус дезинфекция,Розлив
335,2024-07-30 03:37:00,2024-07-30 06:06:00,149.450000,Вмешательство оператора\n,Переналадки,Переход через щ. СИП,Продукт СИП щ. продукт,2214201,переход,Розлив
336,2024-07-30 10:24:00,2024-07-30 10:34:00,9.950000,Вмешательство оператора\n,Переналадки,Смена этикетки,Смена дизайна (экспорт/промо),2214601,Смена банки,Розлив
339,2024-07-30 13:49:00,2024-07-30 14:38:00,48.950000,Вмешательство оператора\n,Санитарные регламенты,Внешние мойки,АПМ (Пенная санитарная обработка),2310503,АПМ,Розлив


В данных за июль не оказалось полезной информации, переходим к датасету поломок.

In [5]:
df = pd.read_excel("Категоризированные поломки.xlsx")
df["reason_short"].value_counts()

reason_short
Замена тормозной пластины           140
Замена дросселя                      70
Замена двух дросселей                57
Устранение недоливов                 48
Замена сбросного дросселя            28
Замена нагнетательного дросселя      23
Иные поломки                         21
Замена наполнителя                   20
Замена дросселя CO2                  19
Замена верхнего дросселя             15
Замена прижимного колпака            14
Замена прижимного колокола           14
Замена нижнего дросселя              13
Замена дросселя нагрузки             13
Иные серьезные поломки               11
Замена прижимного стакана             9
Замена датчика                        9
КИП.МЕХ. Сбой пластичной смазки.      7
Name: count, dtype: int64

Сгенерируем признаки и таргет предсказания следующим образом:

* Сгенерируем временные метки с 2020 по 2024 года с разницей в 3 часа - объекты нашей выборки

* Для каждой временной метки посчитаем - сколько раз за последние X дней случалась ошибка Y, случалась ли она, и суммарное время простоев из-за ошибки. X берем из заранее заготовленного списка временных окон, которые нас интересуют - полгода, 3 месяца, месяц, 2 недели, неделя, 3 дня и один день.

* Тоже самое делаем для таргета, но теперь временные окна отложены не назад, а вперед. Также добавляем признак - какая ошибка будет следующей. 

На этих данных будем обучать нашу модель.

In [47]:
import pandas as pd

def generate_features_and_targets(df, time_range, window_sizes):
    # Создаем новые DataFrames для хранения признаков и таргетов
    feature_rows = []  # Список для хранения строк признаков
    target_rows = []   # Список для хранения строк таргетов
    
    for now in time_range:
        # Создаем строку для текущего времени
        new_feature_row = {'date': now}  # Заполняем временные рамки
        new_target_row = {'date': now}    # Заполняем временные рамки для таргетов
        
        for window in window_sizes:
            start_window = now - pd.Timedelta(days=window)
            
            for reason in df['reason_short'].unique():
                # Генерируем новые признаки
                if window > 0:
                    # Фильтруем строки по текущему reason_short и временным границам
                    mask = (df['reason_short'] == reason) & (df['start'] < now) & (df['end'] > start_window)
                    filtered_df = df[mask]
                    
                    new_feature_row[f'reason_{reason}_within_last_{window}_days'] = int(not filtered_df.empty)
                    new_feature_row[f'reason_{reason}_count_last_{window}_days'] = len(filtered_df)
                    new_feature_row[f'reason_{reason}_duration_sum_last_{window}_days'] = filtered_df['duration'].sum() if not filtered_df.empty else 0
                
                # Генерируем таргеты только для отрицательных окон
                if window < 0:
                    mask = (df['reason_short'] == reason) & (df['start'] > now) & (df['start'] < start_window)
                    filtered_df = df[mask]
                    
                    new_target_row[f'reason_{reason}_within_next_{-window}_days'] = int(not filtered_df.empty)
                    new_target_row[f'reason_{reason}_count_next_{-window}_days'] = len(filtered_df)
                    new_target_row[f'reason_{reason}_duration_sum_next_{-window}_days'] = filtered_df['duration'].sum() if not filtered_df.empty else 0
            
        new_target_row["next_breakdown"] = df[df["start"] > now].iloc[0]["reason_short"]
        
        # Добавляем новую строку в список признаков
        feature_rows.append(new_feature_row)
        
        target_rows.append(new_target_row)

    # Создаем DataFrame из списков строк
    feature_df = pd.DataFrame(feature_rows)
    target_df = pd.DataFrame(target_rows)

    return feature_df, target_df

# Устанавливаем начальную и конечную даты
start_date = '2020-07-01 08:00'
end_date = '2024-10-21 20:00'

# Генерация данных с частотой в 3 часа
time_range = pd.date_range(start=start_date, end=end_date, freq='3h')

feature_data, target_data = generate_features_and_targets(df, time_range, [180, 90, 30, 14, 7, 3, 1, -1, -3, -7, -14, -30])

C:\Users\user\AppData\Local\Temp\ipykernel_11964\3245234933.py:54: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_range = pd.date_range(start=start_date, end=end_date, freq='3H')


In [48]:
feature_data.sample(10)

,date,reason_Замена двух дросселей_within_last_180_days,reason_Замена двух дросселей_count_last_180_days,reason_Замена двух дросселей_duration_sum_last_180_days,reason_Иные поломки_within_last_180_days,reason_Иные поломки_count_last_180_days,reason_Иные поломки_duration_sum_last_180_days,reason_Замена нагнетательного дросселя_within_last_180_days,reason_Замена нагнетательного дросселя_count_last_180_days,reason_Замена нагнетательного дросселя_duration_sum_last_180_days,...,reason_КИП.МЕХ. Сбой пластичной смазки._duration_sum_last_1_days,reason_Замена верхнего дросселя_within_last_1_days,reason_Замена верхнего дросселя_count_last_1_days,reason_Замена верхнего дросселя_duration_sum_last_1_days,reason_Замена наполнителя_within_last_1_days,reason_Замена наполнителя_count_last_1_days,reason_Замена наполнителя_duration_sum_last_1_days,reason_Замена нижнего дросселя_within_last_1_days,reason_Замена нижнего дросселя_count_last_1_days,reason_Замена нижнего дросселя_duration_sum_last_1_days
3602,2021-09-24 14:00:00,1,8,39.683333,0,0,0.000000,1,2,5.066667,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
11245,2024-05-06 23:00:00,0,0,0.000000,0,0,0.000000,0,0,0.000000,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
6337,2022-09-01 11:00:00,1,4,16.733333,1,5,33.950000,1,5,12.483333,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
2971,2021-07-07 17:00:00,1,1,8.633333,1,1,8.583333,1,2,5.066667,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
4516,2022-01-16 20:00:00,1,12,59.233333,0,0,0.000000,1,1,2.150000,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
8214,2023-04-24 02:00:00,1,10,36.416667,0,0,0.000000,1,3,10.800000,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
7787,2023-03-01 17:00:00,1,11,43.733333,1,2,11.333333,1,2,6.633333,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
5359,2022-05-02 05:00:00,1,12,52.283333,1,4,28.683333,1,7,18.150000,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
6631,2022-10-08 05:00:00,0,0,0.000000,1,1,5.266667,0,0,0.000000,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
7021,2022-11-25 23:00:00,1,2,10.700000,1,2,11.333333,0,0,0.000000,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0


In [49]:
target_data.sample(10)

,date,reason_Замена двух дросселей_within_next_1_days,reason_Замена двух дросселей_count_next_1_days,reason_Замена двух дросселей_duration_sum_next_1_days,reason_Иные поломки_within_next_1_days,reason_Иные поломки_count_next_1_days,reason_Иные поломки_duration_sum_next_1_days,reason_Замена нагнетательного дросселя_within_next_1_days,reason_Замена нагнетательного дросселя_count_next_1_days,reason_Замена нагнетательного дросселя_duration_sum_next_1_days,...,reason_Замена верхнего дросселя_within_next_30_days,reason_Замена верхнего дросселя_count_next_30_days,reason_Замена верхнего дросселя_duration_sum_next_30_days,reason_Замена наполнителя_within_next_30_days,reason_Замена наполнителя_count_next_30_days,reason_Замена наполнителя_duration_sum_next_30_days,reason_Замена нижнего дросселя_within_next_30_days,reason_Замена нижнего дросселя_count_next_30_days,reason_Замена нижнего дросселя_duration_sum_next_30_days,next_breakdown
5237,2022-04-16 23:00:00,0,0,0.0,0,0,0.0,0,0,0.0,...,1,1,4.883333,0,0,0.000000,0,0,0.000000,Устранение недоливов
11658,2024-06-27 14:00:00,0,0,0.0,0,0,0.0,0,0,0.0,...,0,0,0.000000,0,0,0.000000,0,0,0.000000,Замена датчика
8630,2023-06-15 02:00:00,0,0,0.0,0,0,0.0,0,0,0.0,...,0,0,0.000000,1,1,6.566667,0,0,0.000000,Иные поломки
5481,2022-05-17 11:00:00,0,0,0.0,0,0,0.0,0,0,0.0,...,0,0,0.000000,0,0,0.000000,0,0,0.000000,Замена тормозной пластины
2000,2021-03-08 08:00:00,0,0,0.0,0,0,0.0,0,0,0.0,...,0,0,0.000000,0,0,0.000000,0,0,0.000000,Замена дросселя CO2
3565,2021-09-19 23:00:00,0,0,0.0,0,0,0.0,0,0,0.0,...,1,1,2.966667,1,2,21.883333,1,1,3.516667,Замена двух дросселей
9106,2023-08-13 14:00:00,0,0,0.0,0,0,0.0,0,0,0.0,...,0,0,0.000000,0,0,0.000000,0,0,0.000000,Замена тормозной пластины
9616,2023-10-16 08:00:00,0,0,0.0,0,0,0.0,0,0,0.0,...,0,0,0.000000,0,0,0.000000,0,0,0.000000,Замена тормозной пластины
9413,2023-09-20 23:00:00,0,0,0.0,0,0,0.0,0,0,0.0,...,0,0,0.000000,0,0,0.000000,0,0,0.000000,Замена тормозной пластины
7222,2022-12-21 02:00:00,0,0,0.0,0,0,0.0,0,0,0.0,...,0,0,0.000000,0,0,0.000000,0,0,0.000000,Устранение недоливов


In [18]:
feature_data.to_excel("Features_3H.xlsx", index=False)
target_data.to_excel("Target_3H.xlsx", index=False)

In [2]:
feature_data = pd.read_excel("Features_3H.xlsx")
target_data = pd.read_excel("Target_3H.xlsx")

Для начала попробуем предсказывать следующую ошибку. Для начала нам нужно отобрать только данные по самым частым ошибкам, чтобы они встречались при обучении:

In [108]:
target_data["next_breakdown"].value_counts()

next_breakdown
Замена тормозной пластины           2928
Устранение недоливов                1648
Замена двух дросселей               1145
Замена дросселя                     1141
Замена сбросного дросселя           1009
Иные поломки                         811
Замена дросселя CO2                  682
Замена датчика                       630
Замена верхнего дросселя             622
Замена дросселя нагрузки             405
Замена нагнетательного дросселя      347
КИП.МЕХ. Сбой пластичной смазки.     341
Иные серьезные поломки               303
Замена прижимного стакана            210
Замена прижимного колокола           126
Замена наполнителя                   116
Замена нижнего дросселя              108
Замена прижимного колпака             17
Name: count, dtype: int64

Выберем четыре самых популярных значения

In [3]:
top_breakdowns = target_data["next_breakdown"].value_counts().nlargest(4).index
target_data_red = target_data[target_data["next_breakdown"].isin(top_breakdowns)]

feature_data_red = feature_data.loc[feature_data.index.isin(target_data_red.index)]

In [20]:
# Загрузка данных
X = feature_data_red.drop(columns=["date"])

# Преобразуем события в числовые метки
event_encoder = LabelEncoder()
y = event_encoder.fit_transform(target_data_red["next_breakdown"])

# Разделение данных на обучающую и тестовую выборки
train_size = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:3 * train_size // 4], X[3 * train_size // 4:train_size], y[:3 * train_size // 4], y[3 * train_size // 4:train_size]

Теперь обучим LGBM предсказывать будущие ошибки и вероятность для них.

In [21]:
# Преобразование y в бинарный формат
lb = LabelBinarizer()
y_train_binarized = lb.fit_transform(y_train)
y_test_binarized = lb.transform(y_test)

# Функция для Optuna
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 10, 50)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1)

    model = lgb.LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, random_state=239, verbose=-1)
    
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:]
    return roc_auc_score(y_test_binarized, y_pred_proba, multi_class='ovr')

study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective, n_trials=50)

# Лучшие параметры
best_params = study.best_params
print(f'Best ROC-AUC: {study.best_value:.2f}')
print(f'Best parameters: {best_params}')

# Обучение модели с лучшими параметрами
model = lgb.LGBMClassifier(**best_params, random_state=42)

model.fit(X_train, y_train)


[I 2025-02-24 22:37:05,917] A new study created in memory with name: no-name-4e35a65c-fb8e-4cce-b552-db66c2efa5a3
[I 2025-02-24 22:37:07,058] Trial 0 finished with value: 0.6278109010219214 and parameters: {'n_estimators': 162, 'max_depth': 32, 'learning_rate': 0.07767233632669933}. Best is trial 0 with value: 0.6278109010219214.
[I 2025-02-24 22:37:08,132] Trial 1 finished with value: 0.6751681816418477 and parameters: {'n_estimators': 144, 'max_depth': 25, 'learning_rate': 0.04460921253867092}. Best is trial 1 with value: 0.6751681816418477.
[I 2025-02-24 22:37:09,293] Trial 2 finished with value: 0.6492199707533546 and parameters: {'n_estimators': 177, 'max_depth': 34, 'learning_rate': 0.0695619222032171}. Best is trial 1 with value: 0.6751681816418477.
[I 2025-02-24 22:37:10,135] Trial 3 finished with value: 0.6196320228983658 and parameters: {'n_estimators': 116, 'max_depth': 39, 'learning_rate': 0.08617563892074125}. Best is trial 1 with value: 0.6751681816418477.
[I 2025-02-24 2

Best ROC-AUC: 0.71
Best parameters: {'n_estimators': 144, 'max_depth': 27, 'learning_rate': 0.058818887145916925}


LGBMClassifier(learning_rate=0.058818887145916925, max_depth=27,
               n_estimators=144, random_state=42)

Итак, мы получили ROC-AUC score 0.71 на исторических данных, что хорошо для задач такого типа. Приналичии большего количества донных и более унифицированной специализации ошибок можно обучить эту модель заново и получить результаты еще лучше, а также модель сможет предсказывать более широкий спектр ошибок.

Также очень важно предсказывать, случится ли та или иная поломка на предприятии в ближайшее время. Давайте для примера будем предсказывать, случатся ли поломки тормозных пластин (замена тормозных пластин) на горизонте трех дней.

In [22]:
X = feature_data.drop(columns=["date"])

event_encoder2 = LabelEncoder()
y = event_encoder2.fit_transform(target_data["reason_Замена тормозной пластины_within_next_1_days"])

train_size = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:3 * train_size // 4], X[3 * train_size // 4:train_size], y[:3 * train_size // 4], y[3 * train_size // 4:train_size]

In [23]:
from sklearn.metrics import f1_score

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 10, 50)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1)

    model = lgb.LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, random_state=239, verbose=-1)
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    return f1_score(y_test, y_pred, average='weighted')

study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective, n_trials=50)

best_params = study.best_params
print(f'Best F1-score: {study.best_value:.2f}')
print(f'Best parameters: {best_params}')

model = lgb.LGBMClassifier(**best_params, random_state=42)

model.fit(X_train, y_train)

[I 2025-02-24 22:38:01,884] A new study created in memory with name: no-name-75289c73-13c0-4501-8a9c-6869256a04e5
[I 2025-02-24 22:38:02,329] Trial 0 finished with value: 0.8520449898202778 and parameters: {'n_estimators': 138, 'max_depth': 45, 'learning_rate': 0.06091086962618634}. Best is trial 0 with value: 0.8520449898202778.
[I 2025-02-24 22:38:02,568] Trial 1 finished with value: 0.9063113025217816 and parameters: {'n_estimators': 54, 'max_depth': 39, 'learning_rate': 0.040602121965573876}. Best is trial 1 with value: 0.9063113025217816.
[I 2025-02-24 22:38:02,941] Trial 2 finished with value: 0.8617807660598652 and parameters: {'n_estimators': 115, 'max_depth': 28, 'learning_rate': 0.046229710172377834}. Best is trial 1 with value: 0.9063113025217816.
[I 2025-02-24 22:38:03,250] Trial 3 finished with value: 0.8975040375548893 and parameters: {'n_estimators': 80, 'max_depth': 47, 'learning_rate': 0.06539801159662477}. Best is trial 1 with value: 0.9063113025217816.
[I 2025-02-24 

Best F1-score: 0.91
Best parameters: {'n_estimators': 54, 'max_depth': 39, 'learning_rate': 0.040602121965573876}


LGBMClassifier(learning_rate=0.040602121965573876, max_depth=39,
               n_estimators=54, random_state=42)

Мы получили метрику F1 0.91, значит, модель отличным precision и recall и надежна в предсказании будущих поломок на горизонте одного дня. Теперь попробуем другие временные горизонты, 3 дня и месяц.

In [24]:
X = feature_data.drop(columns=["date"])

event_encoder2 = LabelEncoder()
y = event_encoder2.fit_transform(target_data["reason_Замена тормозной пластины_within_next_3_days"])

train_size = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:3 * train_size // 4], X[3 * train_size // 4:train_size], y[:3 * train_size // 4], y[3 * train_size // 4:train_size]

In [25]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 10, 50)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1)

    model = lgb.LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, random_state=239, verbose=-1)
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    return f1_score(y_test, y_pred, average='weighted')

study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective, n_trials=50)

best_params = study.best_params
print(f'Best F1-score: {study.best_value:.2f}')
print(f'Best parameters: {best_params}')

model = lgb.LGBMClassifier(**best_params, random_state=42)

model.fit(X_train, y_train)

[I 2025-02-24 22:38:19,800] A new study created in memory with name: no-name-2d33b244-8969-49c0-b7be-0a1269c3adb5
[I 2025-02-24 22:38:20,370] Trial 0 finished with value: 0.6850198808687095 and parameters: {'n_estimators': 188, 'max_depth': 47, 'learning_rate': 0.07184546829475326}. Best is trial 0 with value: 0.6850198808687095.
[I 2025-02-24 22:38:20,782] Trial 1 finished with value: 0.6950571543619661 and parameters: {'n_estimators': 128, 'max_depth': 32, 'learning_rate': 0.04601272239306453}. Best is trial 1 with value: 0.6950571543619661.
[I 2025-02-24 22:38:21,169] Trial 2 finished with value: 0.7408823296433891 and parameters: {'n_estimators': 123, 'max_depth': 42, 'learning_rate': 0.016073335281810312}. Best is trial 2 with value: 0.7408823296433891.
[I 2025-02-24 22:38:21,635] Trial 3 finished with value: 0.6359919762926697 and parameters: {'n_estimators': 142, 'max_depth': 30, 'learning_rate': 0.08554749875582336}. Best is trial 2 with value: 0.7408823296433891.
[I 2025-02-24

Best F1-score: 0.76
Best parameters: {'n_estimators': 51, 'max_depth': 32, 'learning_rate': 0.0369811391070846}


LGBMClassifier(learning_rate=0.0369811391070846, max_depth=32, n_estimators=51,
               random_state=42)

In [26]:
X = feature_data.drop(columns=["date"])

event_encoder2 = LabelEncoder()
y = event_encoder2.fit_transform(target_data["reason_Замена тормозной пластины_within_next_30_days"])

train_size = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:3 * train_size // 4], X[3 * train_size // 4:train_size], y[:3 * train_size // 4], y[3 * train_size // 4:train_size]

In [27]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 10, 50)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1)

    model = lgb.LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, random_state=239, verbose=-1)
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    return f1_score(y_test, y_pred, average='weighted')

study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective, n_trials=50)

best_params = study.best_params
print(f'Best F1-score: {study.best_value:.2f}')
print(f'Best parameters: {best_params}')

model = lgb.LGBMClassifier(**best_params, random_state=42)

model.fit(X_train, y_train)

[I 2025-02-24 22:38:38,284] A new study created in memory with name: no-name-6857c6f3-2342-4133-8514-629c8520e74a
[I 2025-02-24 22:38:38,644] Trial 0 finished with value: 0.648133760176787 and parameters: {'n_estimators': 120, 'max_depth': 11, 'learning_rate': 0.025362619782957466}. Best is trial 0 with value: 0.648133760176787.
[I 2025-02-24 22:38:39,112] Trial 1 finished with value: 0.6365730763600842 and parameters: {'n_estimators': 170, 'max_depth': 16, 'learning_rate': 0.045852765346850524}. Best is trial 0 with value: 0.648133760176787.
[I 2025-02-24 22:38:39,531] Trial 2 finished with value: 0.666236624394249 and parameters: {'n_estimators': 133, 'max_depth': 33, 'learning_rate': 0.08139795833396328}. Best is trial 2 with value: 0.666236624394249.
[I 2025-02-24 22:38:39,970] Trial 3 finished with value: 0.6365730763600842 and parameters: {'n_estimators': 138, 'max_depth': 29, 'learning_rate': 0.064608440041324}. Best is trial 2 with value: 0.666236624394249.
[I 2025-02-24 22:38:

Best F1-score: 0.67
Best parameters: {'n_estimators': 133, 'max_depth': 33, 'learning_rate': 0.08139795833396328}


LGBMClassifier(learning_rate=0.08139795833396328, max_depth=33,
               n_estimators=133, random_state=42)

На 3 днях мы получили F1 0.76, а на месяце - 0.67. Это показывает, что модель имеет относительно хорошие precision и recall как на маленьких, так и на больших поломках, что делает ее качественным решением для предсказывания будущих поломок. Также можно заметить, что модель была обучена достаточно на узком количестве признаков. При дополнительном включении данных о показателях датчиков за весь временной горизонт (5 лет), качество модели можно сильно улучшить, если собирать датасет аналогично тому принципу, который продемонстрирован в данном ноутбуке, ведь датчики содержат важную информацию, позволяющую предвидеть скорую поломку.

Однако, это ещё не всё! С помощью библиотеки OpenFE мы можем дополнительно генерировать признаки, что поможет улучшить качество предсказаний, сделаем это.

In [157]:
!pip install --upgrade openfe

   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.3 MB ? eta -:--:--
   -- ------------------------------------- 1.6/25.3 MB 4.9 MB/s eta 0:00:05
   ---- ----------------------------------- 2.9/25.3 MB 5.8 MB/s eta 0:00:04
   ------- -------------------------------- 4.5/25.3 MB 6.3 MB/s eta 0:00:04
   --------- ------------------------------ 6.0/25.3 MB 6.5 MB/s eta 0:00:03
   ------------ --------------------------- 7.6/25.3 MB 6.7 MB/s eta 0:00:03
   ------------ --------------------------- 8.1/25.3 MB 6.1 MB/s eta 0:00:03
   ------------- -------------------------- 8.7/25.3 MB 5.5 MB/s eta 0:00:04
   -------------- ------------------------- 9.2/25.3 MB 5.4 MB/s eta 0:00:03
   -------------- ------------------------- 9.4/25.3 MB 5.0 MB/s eta 0:00:04
   --------------- ------------------------ 9.7/25.3 MB 4.4 MB/s eta 0:00:04
   --------------- ------------------------ 10.0/25.3 MB 4.1 MB/s eta 0:00:04
   ---------


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Вернемся к задаче предсказания следующей поломки. Признаков в нашем датасете слишком много, и генерация новых признаков на их основе займет крайне большое время. Поэтому для начала мы проведем отбор нужных признаков, используя встроенные функции LGBM.

In [4]:
X = feature_data_red.drop(columns=["date"])

event_encoder = LabelEncoder()
y = event_encoder.fit_transform(target_data_red["next_breakdown"])

train_size = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:3 * train_size // 4], X[3 * train_size // 4:train_size], y[:3 * train_size // 4], y[3 * train_size // 4:train_size]

In [5]:
# Преобразование y в бинарный формат
lb = LabelBinarizer()
y_train_binarized = lb.fit_transform(y_train)
y_test_binarized = lb.transform(y_test)

# Функция для Optuna
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 10, 50)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1)

    model = lgb.LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, random_state=239, verbose=-1)
    
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:]
    return roc_auc_score(y_test_binarized, y_pred_proba, multi_class='ovr')

study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective, n_trials=50)

# Лучшие параметры
best_params = study.best_params
print(f'Best ROC-AUC: {study.best_value:.2f}')
print(f'Best parameters: {best_params}')

# Обучение модели с лучшими параметрами
model = lgb.LGBMClassifier(**best_params, random_state=42)

model.fit(X_train, y_train)


[I 2025-02-24 22:52:55,069] A new study created in memory with name: no-name-83b66acc-201b-425a-8365-8a76344d5dbb
[I 2025-02-24 22:52:56,239] Trial 0 finished with value: 0.6100744440637247 and parameters: {'n_estimators': 151, 'max_depth': 41, 'learning_rate': 0.07299067516138343}. Best is trial 0 with value: 0.6100744440637247.
[I 2025-02-24 22:52:57,190] Trial 1 finished with value: 0.6976607419058065 and parameters: {'n_estimators': 132, 'max_depth': 17, 'learning_rate': 0.04784914485525552}. Best is trial 1 with value: 0.6976607419058065.
[I 2025-02-24 22:52:58,152] Trial 2 finished with value: 0.6365727128126515 and parameters: {'n_estimators': 137, 'max_depth': 19, 'learning_rate': 0.05144566198563247}. Best is trial 1 with value: 0.6976607419058065.
[I 2025-02-24 22:52:59,012] Trial 3 finished with value: 0.6823752303468222 and parameters: {'n_estimators': 115, 'max_depth': 29, 'learning_rate': 0.04687162284016382}. Best is trial 1 with value: 0.6976607419058065.
[I 2025-02-24 

Best ROC-AUC: 0.71
Best parameters: {'n_estimators': 198, 'max_depth': 10, 'learning_rate': 0.02170643272226406}


LGBMClassifier(learning_rate=0.02170643272226406, max_depth=10,
               n_estimators=198, random_state=42)

Теперь отберем нужные параметры.

In [6]:
feature_importances = model.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Отберем 50 самых важных признаков
feature_importance_df_selected = feature_importance_df[:50]
feature_importance_df_selected.head()

,Feature,Importance
10,reason_Замена тормозной пластины_count_last_18...,995
65,reason_Замена тормозной пластины_duration_sum_...,691
11,reason_Замена тормозной пластины_duration_sum_...,662
1,reason_Замена двух дросселей_count_last_180_days,617
55,reason_Замена двух дросселей_count_last_90_days,597


In [7]:
X = feature_data_red.drop(columns=["date"])[feature_importance_df_selected["Feature"]]

event_encoder = LabelEncoder()
y = event_encoder.fit_transform(target_data_red["next_breakdown"])

train_size = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:3 * train_size // 4], X[3 * train_size // 4:train_size], y[:3 * train_size // 4], y[3 * train_size // 4:train_size]

Теперь мы можем генерировать новые признаки!

In [ ]:
from openfe import OpenFE, transform

ofe = OpenFE()
features = ofe.fit(data=X_train, label=y_train)

The number of candidate features is 25939
Start stage I selection.


100%|██████████| 4/4 [12:58<00:00, 194.57s/it]


6534 same features have been deleted.


100%|██████████| 4/4 [07:08<00:00, 107.08s/it]


The number of remaining candidate features is 2425
Start stage II selection.


100%|██████████| 4/4 [00:09<00:00,  2.43s/it]


Finish data processing.


TypeError: transform() missing 1 required positional argument: 'n_jobs'

In [10]:
train_x, test_x = transform(X_train, X_test, features, n_jobs=8)

In [12]:
train_x

,reason_Замена тормозной пластины_count_last_180_days,reason_Замена тормозной пластины_duration_sum_last_90_days,reason_Замена тормозной пластины_duration_sum_last_180_days,reason_Замена двух дросселей_count_last_180_days,reason_Замена двух дросселей_count_last_90_days,reason_Замена тормозной пластины_duration_sum_last_30_days,reason_Замена двух дросселей_duration_sum_last_90_days,reason_Устранение недоливов_count_last_180_days,reason_Замена дросселя_count_last_90_days,reason_Замена дросселя_duration_sum_last_30_days,...,autoFE_f_2415,autoFE_f_2416,autoFE_f_2417,autoFE_f_2418,autoFE_f_2419,autoFE_f_2420,autoFE_f_2421,autoFE_f_2422,autoFE_f_2423,autoFE_f_2424
openfe_index,,,,,,,,,,,,,,,,,,,,,
4,15,42.216667,66.866667,3,2,10.400000,7.633333,0,2,9.333333,...,17.933333,0.000000,42.216667,42.216667,42.216667,5.633333,0.507576,0.0,42.216667,0.672205
5,15,42.216667,66.866667,3,2,10.400000,7.633333,0,2,9.333333,...,17.933333,0.000000,42.216667,42.216667,42.216667,5.633333,0.507576,0.0,42.216667,0.672205
6,15,42.216667,66.866667,3,2,10.400000,7.633333,0,2,9.333333,...,17.933333,0.000000,42.216667,42.216667,42.216667,5.633333,0.507576,0.0,42.216667,0.672205
7,15,42.216667,66.866667,3,2,10.400000,7.633333,0,2,9.333333,...,17.933333,0.000000,42.216667,42.216667,42.216667,5.633333,0.507576,0.0,42.216667,0.672205
8,15,42.216667,66.866667,3,2,10.400000,7.633333,0,2,9.333333,...,17.933333,0.000000,42.216667,42.216667,42.216667,5.633333,0.507576,0.0,42.216667,0.672205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7996,21,31.116667,89.466667,11,5,17.766667,19.933333,3,6,9.516667,...,15.800000,13.983333,31.116667,31.116667,31.116667,17.933333,0.869744,22.0,45.100000,0.618071
7997,21,31.116667,89.466667,11,5,17.766667,19.933333,3,6,9.516667,...,15.800000,13.983333,31.116667,31.116667,31.116667,17.933333,0.869744,22.0,45.100000,0.618071
7998,21,31.116667,89.466667,11,5,17.766667,19.933333,3,6,9.516667,...,15.800000,13.983333,31.116667,31.116667,31.116667,17.933333,0.869744,22.0,45.100000,0.618071


Обучим на этих данных нашу модель.

In [14]:
# Преобразование y в бинарный формат
lb = LabelBinarizer()
y_train_binarized = lb.fit_transform(y_train)
y_test_binarized = lb.transform(y_test)

# Функция для Optuna
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 10, 50)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1)

    model = lgb.LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, random_state=239, verbose=-1)
    
    model.fit(train_x, y_train)
    y_pred_proba = model.predict_proba(test_x)[:]
    return roc_auc_score(y_test_binarized, y_pred_proba, multi_class='ovr')

study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective, n_trials=15)

# Лучшие параметры
best_params = study.best_params
print(f'Best ROC-AUC: {study.best_value:.2f}')
print(f'Best parameters: {best_params}')

# Обучение модели с лучшими параметрами
model = lgb.LGBMClassifier(**best_params, random_state=42)

model.fit(train_x, y_train)


[I 2025-02-24 23:43:00,160] A new study created in memory with name: no-name-e26a8564-737c-4db3-a590-4f092bffb096
[I 2025-02-24 23:43:16,124] Trial 0 finished with value: 0.8252490505010961 and parameters: {'n_estimators': 143, 'max_depth': 19, 'learning_rate': 0.08716300415073008}. Best is trial 0 with value: 0.8252490505010961.
[I 2025-02-24 23:43:40,941] Trial 1 finished with value: 0.8888148574904848 and parameters: {'n_estimators': 197, 'max_depth': 29, 'learning_rate': 0.05001101922618978}. Best is trial 1 with value: 0.8888148574904848.
[I 2025-02-24 23:44:04,061] Trial 2 finished with value: 0.7998568692371362 and parameters: {'n_estimators': 171, 'max_depth': 20, 'learning_rate': 0.04131719240064681}. Best is trial 1 with value: 0.8888148574904848.
[I 2025-02-24 23:44:14,268] Trial 3 finished with value: 0.5107487784275831 and parameters: {'n_estimators': 70, 'max_depth': 21, 'learning_rate': 0.0424683670930599}. Best is trial 1 with value: 0.8888148574904848.
[I 2025-02-24 23

Best ROC-AUC: 0.89
Best parameters: {'n_estimators': 197, 'max_depth': 29, 'learning_rate': 0.05001101922618978}


LGBMClassifier(learning_rate=0.05001101922618978, max_depth=29,
               n_estimators=197, random_state=42)

Попробуем то же самое сделать и для предсказаний поломок!

In [ ]:
X = feature_data.drop(columns=["date"])

event_encoder2 = LabelEncoder()
y = event_encoder2.fit_transform(target_data["reason_Замена тормозной пластины_within_next_3_days"])

train_size = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:3 * train_size // 4], X[3 * train_size // 4:train_size], y[:3 * train_size // 4], y[3 * train_size // 4:train_size]